<a href="https://colab.research.google.com/github/beknur/NLP/blob/master/Hypernyms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=12488a0a2e90277289c7363fd3180ca83a2972a11ff2d3c5397d52fb51161480
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


### Lab 2: Hyponyms and Hypernyms

In [59]:
import pandas as pd
import wikipedia
import os
from tqdm import tqdm
from joblib import Parallel, delayed
import multiprocessing
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from ipywidgets import interact, interactive, fixed, interact_manual,widgets
from IPython.display import display
import json
import urllib.request
import time
from bs4 import BeautifulSoup
from urllib.parse import quote
num_cores = multiprocessing.cpu_count()
print(num_cores)
wikipedia.set_lang("ru")
# DATA_PATH_LIST = ['D:','src2','taxonomy-enrichment','data','training_data']
DATA_PATH_LIST = ['.']
EMBEDDING_MODEL_FILENAME = "wiki_node2vec.bin"
DATA_PATH="training_nouns.tsv"
df = pd.read_csv("training_nouns.tsv",sep='\t')
print(df)
%matplotlib inline

2
      SYNSET_ID  ...                                       PARENT_TEXTS
0      126551-N  ...  ['СОВЕРШЕННОЛЕТНИЕ ЛИЦА, СОВЕРШЕННОЛЕТНИЙ ГРАЖ...
1      126551-N  ...  ['ВЕРУЮЩИЙ В ХРИСТА, БРАТЬЯ ВО ХРИСТЕ, ХРИСТИА...
2      120440-N  ...  ['ВЫРАЖЕНИЕ ЛИЦА, ВЫРАЖЕНИЕ, МИНА, ВЫРАЖЕНИЕ Н...
3      120440-N  ...  ['ПРЕПОДНЕСЕНИЕ, ПРЕДСТАВЛЕНИЕ', 'ОСМЕИВАНИЕ, ...
4        1277-N  ...  ['БУХГАЛТЕРСКИЙ ДОКУМЕНТ, ДОКУМЕНТ БУХГАЛТЕРСК...
...         ...  ...                                                ...
17237  131787-N  ...  ['НЕДОМОГАНИЕ, ЗАБОЛЕВАНИЕ ЧЕЛОВЕКА, ЗАБОЛЕВАН...
17238  142357-N  ...  ['НЕДОМОГАНИЕ, ЗАБОЛЕВАНИЕ ЧЕЛОВЕКА, ЗАБОЛЕВАН...
17239  144031-N  ...  ['НЕДОМОГАНИЕ, ЗАБОЛЕВАНИЕ ЧЕЛОВЕКА, ЗАБОЛЕВАН...
17240  147153-N  ...  ['НЕДОМОГАНИЕ, ЗАБОЛЕВАНИЕ ЧЕЛОВЕКА, ЗАБОЛЕВАН...
17241  153553-N  ...  ['НЕДОМОГАНИЕ, ЗАБОЛЕВАНИЕ ЧЕЛОВЕКА, ЗАБОЛЕВАН...

[17242 rows x 4 columns]


In [0]:
def prestr(x):
    return str(x).replace('\"','').replace("'",'"')

In [0]:
class DefDict(defaultdict):
    def __missing__(self, key):
        self[key] = key
        return key
    
idx2syns = DefDict(lambda x:x)
for val in df.values:
    idx2syns[val[0]]=val[1]
    try:
        pidxs = json.loads(prestr(val[2]))
        concp = [el.split(",")[0] for el in json.loads(prestr(val[3]))]
        idx2syns.update(dict(zip(pidxs,concp)))
    except:
        print(prestr(val[2]))
        print(prestr(val[3]))

### Interactive visualization of hyponyms and hypernyms

In [8]:
!apt-get -qq install -y graphviz libgraphviz-dev graphviz-dev pkg-config

Selecting previously unselected package libgtk2.0-common.
(Reading database ... 134443 files and directories currently installed.)
Preparing to unpack .../0-libgtk2.0-common_2.24.32-1ubuntu1_all.deb ...
Unpacking libgtk2.0-common (2.24.32-1ubuntu1) ...
Selecting previously unselected package libgtk2.0-0:amd64.
Preparing to unpack .../1-libgtk2.0-0_2.24.32-1ubuntu1_amd64.deb ...
Unpacking libgtk2.0-0:amd64 (2.24.32-1ubuntu1) ...
Selecting previously unselected package libgail18:amd64.
Preparing to unpack .../2-libgail18_2.24.32-1ubuntu1_amd64.deb ...
Unpacking libgail18:amd64 (2.24.32-1ubuntu1) ...
Selecting previously unselected package libgail-common:amd64.
Preparing to unpack .../3-libgail-common_2.24.32-1ubuntu1_amd64.deb ...
Unpacking libgail-common:amd64 (2.24.32-1ubuntu1) ...
Selecting previously unselected package libxdot4.
Preparing to unpack .../4-libxdot4_2.40.1-2_amd64.deb ...
Unpacking libxdot4 (2.40.1-2) ...
Selecting previously unselected package libgvc6-plugins-gtk.
Prep

In [9]:
pip install pygraphviz

     |████████████████████████████████| 122kB 4.8MB/s 
  Created wheel for pygraphviz: filename=pygraphviz-1.5-cp36-cp36m-linux_x86_64.whl size=157577 sha256=74183a249a4c82c453bddf4472c8c07f6654bc362a324af9b968ec3a4b932c33
  Stored in directory: /root/.cache/pip/wheels/65/54/69/1aee9e66ab19916293208d4c9de0d3898adebe6b2eeff6476b
Successfully built pygraphviz


In [11]:
button = widgets.Button(description="Draw")
query = widgets.Text(
    value='ДОКУМЕНТ',
    placeholder='Query',
    description='String:',
    disabled=False
)
display(button,query)




def creategraph(df):
    res = []
    for row in df.values:
        cohyps = row[1].split(",")
        for idx,cohyp in enumerate(cohyps):
            for parent in json.loads(prestr(row[2])):
                res.append((row[0]+'-'+str(idx),parent))
    return res

def graphdraw(b):
    print("graphdraw",query.value)
    subset = df[df['TEXT'].str.contains(query.value.upper())]
    g = nx.DiGraph()
    for el in subset.values:
        cohyps = el[1].split(",")
        print(cohyps)
        syns = idx2syns[el[0]]
        for child in cohyps:
            for parent in json.loads(prestr(el[2])):
                ed = g.add_edge(child,idx2syns[parent],label="is a")
    plt.figure(figsize=(15,15))
    pos = nx.nx_agraph.graphviz_layout(g)
    nx.draw(g,with_labels=True,pos=pos)
#     edge_labels=nx.draw_networkx_edge_labels(g,pos=pos)
    plt.show()
button.on_click(graphdraw)

Button(description='Draw', style=ButtonStyle())

Text(value='ДОКУМЕНТ', description='String:', placeholder='Query')

### Pattern extractor

Yargy — библиотека для извлечения структурированной информации из текстов на русском языке. Правила описываются контекстно-свободными грамматиками и словарями ключевых слов. Банк готовых правил для имён, дат, адресов и других сущностей доступен в репозитории Natasha.
* https://yargy.readthedocs.io/ru/latest/
* http://pymorphy2.readthedocs.io/en/latest/user/grammemes.html
* https://github.com/natasha/natasha

### Токенизатор

In [12]:
pip install yargy

     |████████████████████████████████| 51kB 3.4MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 7.1MB 15.2MB/s 


In [13]:
from yargy.tokenizer import MorphTokenizer


tokenizer = MorphTokenizer()
text = '''Ростов-на-Дону
Длительностью 18ч. 10мин.
Яндекс.Такси
π ≈ 3.1415
1 500 000$
http://vk.com
'''
for line in text.splitlines():
    print([_.value for _ in tokenizer(line)])

['Ростов', '-', 'на', '-', 'Дону']
['Длительностью', '18', 'ч', '.', '10', 'мин', '.']
['Яндекс', '.', 'Такси']
['π', '≈', '3', '.', '1415']
['1', '500', '000', '$']
['http', ':', '/', '/', 'vk', '.', 'com']


# Газеттир
Газеттир нужен для удобной работы с последовательностью слов. Например, можно написать:

In [0]:
from yargy import or_, rule
from yargy.predicates import normalized

RULE = or_(
    rule(normalized('dvd'), '-', normalized('диск')),
    rule(normalized('видео'), normalized('файл'))
)

In [15]:
from yargy import Parser
from yargy.pipelines import morph_pipeline


RULE = morph_pipeline([
    'dvd-диск',
    'видео файл',
    'видеофильм',
    'газета',
    'электронный дневник',
    'эссе',
])

parser = Parser(RULE)
text = 'Видео файл на dvd-диске'
for match in parser.findall(text):
    print([_.value for _ in match.tokens])

['Видео', 'файл']
['dvd', '-', 'диске']


In [16]:
from yargy import Parser, rule, and_
from yargy.predicates import gram, is_capitalized, dictionary


GEO = rule(
    and_(
        gram('ADJF'),  # так помечается прилагательное, остальные пометки описаны в
                       # http://pymorphy2.readthedocs.io/en/latest/user/grammemes.html
        is_capitalized()
    ),
    gram('ADJF').optional().repeatable(),
    dictionary({
        'федерация',
        'республика'
    })
)


parser = Parser(GEO)
text = '''
В Чеченской республике на день рождения ...
Донецкая народная республика провозгласила ...
Башня Федерация — одна из самых высоких ...
'''
for match in parser.findall(text):
    print([_.value for _ in match.tokens])

['Чеченской', 'республике']
['Донецкая', 'народная', 'республика']


### Предикаты

Предикат — функция, которая принимает на вход токен и возвращает True или False. В Yargy встроено много готовых предикатов. Полный список есть в справочнике. Предикаты комбинируются с помощью and_, or_ и not_.

In [17]:
from yargy import and_, not_
from yargy.tokenizer import MorphTokenizer
from yargy.predicates import is_capitalized, eq


tokenizer = MorphTokenizer()
token = next(tokenizer('Стали'))

predicate = is_capitalized()
print(predicate(token))

predicate = and_(
    is_capitalized(),
    not_(eq('марки'))
)
print(predicate(token))

True
True


### Грамматики
В Yargy используется специальный DSL для описания грамматик. Любую контекстно-свободную грамматику можно описать с помощью конструкций Питона. Например, есть примитивная грамматика для размеров одежды:

KEY -> р. | размер

VALUE -> S | M | L

SIZE -> KEY VALUE



In [18]:
from yargy import rule, or_


KEY = or_(
    rule('р', '.'),
    rule('размер')
).named('KEY')
VALUE = or_(
    rule('S'),
    rule('M'),
    rule('L'),
    rule('XS'),
).named('VALUE')
SIZE = rule(
    KEY,
    VALUE
).named('SIZE')
SIZE.normalized.as_bnf

SIZE -> KEY VALUE
KEY -> 'р' '.' | 'размер'
VALUE -> 'S' | 'M' | 'L' | 'XS'


In [19]:
parser = Parser(
    SIZE
)
text = 'размер M; размер A; размер XS;'
for match in parser.findall(text):
    print([_.value for _ in match.tokens])

['размер', 'M']
['размер', 'XS']


In [0]:
from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, is_capitalized, dictionary, eq
import re
import pandas as pd
from tqdm import tqdm_notebook
from gensim import utils

In [0]:

START = rule(
    or_(
        rule(gram('ADJF')),
        rule(gram('NOUN'))
    ).optional(),
    gram('NOUN')
)

START_S = or_(
    eq('такой'),
    eq('такие'),
)

KAK = eq('как')
INCLUDING = or_(
    or_(
        eq('в'),
        eq('том'),
        eq('числе'),
    ),
    eq('включающий'),
    or_(
        eq('включающий'),
        eq('в'),
        eq('себя'),
    ),
    or_(
        eq('включающие'),
        eq('в'),
        eq('себя'),
    ),
    eq('включающие'),
    eq('особенно'),

)

MID_S = or_(
    rule(
        or_(
            eq('такой'),
            eq('такие'),
        ),
        eq('как')
    )
)
ATAKJE = rule(
    eq(','),
    eq('а'),
    eq('также')
)

MID = or_(
    rule(
        eq('это')
    ),
    rule(
        eq('—')
    ),
    rule(
        eq('—'),
        eq('это')
    ),
    rule(
        eq('—'),
        not_(eq('км'))
    ),
    rule(
        or_(
            eq('и'),
            eq('или'),
        ),
        eq('другие')
    )
)

END = or_(
    rule(
        gram('NOUN'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable()
    )
)

Item = fact(
    'Item',
    [attribute('titles').repeatable()]
)


IGNORE = rule(
    '(',
    not_(eq(')')).repeatable(),
    ')'
)

ITEM = rule(
    IGNORE.interpretation(
        Item.titles
    ),
    eq(',').optional() 
).repeatable().interpretation(
    Item
)

In [0]:
def get_hyperonyms(main_word):
    HYPONYM = eq(utils.deaccent(main_word))
    RULE = or_(
        rule(HYPONYM, ATAKJE, START, MID, END),
        rule(HYPONYM, MID, END),
        rule(START_S, END, KAK, HYPONYM),
        rule(END, INCLUDING, HYPONYM)
    )
    parser = Parser(RULE) 
    text = utils.deaccent(wikipedia.summary(main_word))
    print(text)
    text = re.sub(r'\(.+?\)', '', text)
    text = text.lower().replace('* сергии радонежскии* ', '')
    for idx, match in enumerate(parser.findall(text.lower())):
        k = [_.value for _ in match.tokens]
        print(k)

In [61]:
get_hyperonyms("АБСЕНТЕИЗМ")

Абсентеизм (от лат. absens, absentis — отсутствующии, англ. absenteeism) — уклонение избирателеи от участия в голосовании на выборах, или, в более широком понимании, политическое поведение, характеризующееся бездеиствием, то есть уклонением от какого-либо политического участия (электоральное поведение, партииная деятельность, участие в митингах и демонстрациях и т. д.), но главным образом подразумевается уклонение от своих прямых электоральных функции.
Также абсентеизмом называют поведенческую модель, при которои работник систематически отсутствует на рабочем месте и избегает своих обязанностеи. Традиционно абсентеизм рассматривается как индикатор плохои индивидуальнои производительности и как одна из главных проблем менеджмента, исследование которои развивается в экономических и квази-экономических терминах. Последние исследования в этои области сосредоточились на рассмотрении и осмыслении абсентеизма в роли индикатора психологическои, медицинскои и социальнои адаптации к работе.
Наиб

#### Task 1 (deadline 19.02.2020 24:00)
* Find your name on the spreadsheet https://docs.google.com/spreadsheets/d/1RR2I6toCkebbGU1UK83HS70Ru_l0_o-nnZIHyiFB0No/edit?usp=sharing. In opposite of your name there are 24 words of hyponyms, you have to insert five corresponding hypernyms next to them. Examples of hyponyms and hyperonyms relationship you can find above in the current Jupiter notebook.
* Find for each pair of hyponyms and hypernyms a corresponding snippet of a text with their mentions. The source of the text can be any free resources, e.g., Wikipedia, Google, Yandex, others. You should save the snippets and their URLs within the lab2 folder in your NLP git-repo with .csv file-extension in a single file.

#### Task 2 (deadline 26.02.2020 24:00)
* It would be best if you created a pandas DataFrame of the texts from the previous task. And apply to the DataFrame the function 'get_hyperonyms,' which must return the list of the corresponding hypernyms from the text automatically. If there are errors or misses, you should fix them in the code for your case of the 24 words. Nevertheless, it is strictly prohibited to use hard coding. Save your notebook with parser code within the lab2 folder in your NLP git-repo.

In [78]:
words = pd.read_excel('wordsNLPLab2.xlsx', sep='|', encoding='utf-8')

words.head()

,HYPONYM
0,ПРЕСЕРВЫ
1,ПРЕССОВЩИК
2,ПРЕФЕРАНС
3,ПРИБОРИСТ
4,ПРИВОДНЕНИЕ


In [79]:
wiki = "https://ru.wiktionary.org/wiki/"
for word in list(words['HYPONYM']):
    wordConvert = quote(word.lower(),safe='')
    url = wiki + wordConvert
    print("Слово: " + word + " Cсылка: " + url)
    source = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(source)
    span = soup.select_one("#Гиперонимы")
    h4 = span.find_parent('h4')
    ol = h4.find_next_sibling('ol')
    hyponyms = word + " => Гиперонимы => "
    for li in ol.find_all('li'):
      try:
        if li.text:
          hyponyms  += li.text.upper() + ", "
      except: 
        pass
    print(hyponyms.replace(",",";"))
    time.sleep(3.0)

Слово: ПРЕСЕРВЫ Cсылка: https://ru.wiktionary.org/wiki/%D0%BF%D1%80%D0%B5%D1%81%D0%B5%D1%80%D0%B2%D1%8B
ПРЕСЕРВЫ => Гиперонимы => 
Слово: ПРЕССОВЩИК Cсылка: https://ru.wiktionary.org/wiki/%D0%BF%D1%80%D0%B5%D1%81%D1%81%D0%BE%D0%B2%D1%89%D0%B8%D0%BA
ПРЕССОВЩИК => Гиперонимы => 
Слово: ПРЕФЕРАНС Cсылка: https://ru.wiktionary.org/wiki/%D0%BF%D1%80%D0%B5%D1%84%D0%B5%D1%80%D0%B0%D0%BD%D1%81
ПРЕФЕРАНС => Гиперонимы => КАРТОЧНАЯ ИГРА; ИГРА; —; 
Слово: ПРИБОРИСТ Cсылка: https://ru.wiktionary.org/wiki/%D0%BF%D1%80%D0%B8%D0%B1%D0%BE%D1%80%D0%B8%D1%81%D1%82
ПРИБОРИСТ => Гиперонимы => 
Слово: ПРИВОДНЕНИЕ Cсылка: https://ru.wiktionary.org/wiki/%D0%BF%D1%80%D0%B8%D0%B2%D0%BE%D0%B4%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5
ПРИВОДНЕНИЕ => Гиперонимы => 
Слово: ПРИКЛАДНИК Cсылка: https://ru.wiktionary.org/wiki/%D0%BF%D1%80%D0%B8%D0%BA%D0%BB%D0%B0%D0%B4%D0%BD%D0%B8%D0%BA
ПРИКЛАДНИК => Гиперонимы => 
Слово: ПРИКУРИВАТЕЛЬ Cсылка: https://ru.wiktionary.org/wiki/%D0%BF%D1%80%D0%B8%D0%BA%D1%83%D1%80%D0%B8%D0%B2%D0%B0%D1

In [80]:
for word in list(words['HYPONYM']):
    print(word)
    get_hyperonyms(word)

ПРЕСЕРВЫ
Пресервы (от позднелат. praeservo «предохраняю») — рыбные продукты, которые были подвергнуты специальнои обработке (консервации) и упаковке с целью длительного хранения без порчи. Способом консервации является соление — обработка рыбы большим количествам повареннои соли. Кроме этого пресервы обычно содержат антисептик как дополнительныи консервант. 
В процессе посола происходит некоторая ферментация рыбы — рыба «созревает», в результате чего вкус ее значительно меняется, она становится вкуснее.
Пресервы принципиально отличаются от консервов тем, что не проходят тепловои обработки (термическои стерилизации). В связи с этим пресервы требуют строгого соблюдения непрерывности холодильнои цепи с момента изготовления до их потребления. Обычно, температура хранения пресервов составляет от 0 до −8 °C. При этои температуре бактериальные процессы сильно замедляются, но замораживания продукта не происходит (из-за высокого содержания соли). Срок хранения пресервов, в сравнении с консервам

/usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


DisambiguationError: ignored